# Proximal Policy Optimization(PPO)

## 목적

- agent가 환경과 상호작용을 통해 data를 sampling하는 것과 stochastic gradient ascent를 사용하여 surrogate objective function을 optimize하는 것을 번갈아가면서 하는 방법이다. data sample마다 gradient update를 수행하는 기존의 방법과는 달리 mini-batch update가 가능한 objective function이다.

## 설명

- PPO는 TRPO에서 기원한 알고리즘이고, PPO, TRPO는 알고리즘의 학습 안정성을 높이기 위해서 DQN에서 사용했던 target network와 experience replay를 사용한다.
- 또한, network weights를 update하는 것이 너무 많이 update하지 않도록 제한을 두었다.

- 물론, update의 폭을 조절하기 위해서 learning_rate을 조절하기도 하지만 경험적인 조절이 아닌 이론적인 기반의 update rule을 제안하였다.(TRPO)
- TRPO에서는 KL-divergence 방법을 사용하여 objective function에 constraint를 두는 방식으로 진행하였다. 하지만 clipping이 더 좋은 결과를 보였다.

### TRPO, PPO

- TRPO와 PPO 알고리즘은 학습의 안정성을 높이기 위한 기법들이 사용되었다. 기존에 있던 알고리즘들은 weight update가 너무 급격하게 바뀌는 문제점이 있었다.
- 위 문제를 해결하기 위해서 TRPO에서는 KL-divergence를 사용하였고, PPO에서는 clipping을 사용하였다. 결과는 PPO가 더 좋았다.

### 방법

- Clipped probability ratios를 포함하는 objective function 사용
  - TRPO의 data efficiency와 robustness를 유지하면서 1차 미분만 사용함. 2차 미분 사용안하기 때문에 계산량이 줄어듦.
  - Policy 성능에 대한 lower bound를 제공함.

### Policy optimization

- 앞서 언급한 것과 같이 일반적인 policy gradient method는 parameter space에서의 update가 policy space에서 큰 변화를 유발할 수 있다.

### Trust region methods

- Policy update 크기에 대한 constraint를 가지고 objective function(surrogate function)을 최대화하는 것이 목표!
$$ maximize_\theta \;\; \hat{\mathbb{E}}_t[\frac{\pi_\theta(a_t\;|\;s_t)}{\pi_{\theta_{old}}(a_t\;|\;s_t)} \hat{A}_t] \\ subject\;to \;\;\hat{\mathbb{E}}_t[KL[\pi_{\theta_{old}}(\cdot|s_t), \pi_\theta(\cdot|s_t)]] \leq \delta $$

- 위 식의 constraint로 급격한 policy update를 막는다.

- TRPO에서 constraint optimization 문제를 풀기 위해서 아래와 같은 방법이 필요했다.
  - Fisher information matrix인 2차 미분 KL-divergence를 사용하거나,
    - 하지만 너무 많은 계산량을 요구하는 방법이다.
  - Conjugate gradient를 사용한다.
    - 구현이 너무 어렵다.

- 이론적으로는,
$$ maximize_\theta \; \hat{\mathbb{E}}_t \begin{bmatrix} \frac{\pi_\theta(a_t\;|\;s_t)}{\pi_{\theta_{old}}(a_t\;|\;s_t)} \hat{A}_t - \beta KL[\pi_{\theta_{old}} (\cdot | s_t), \pi_\theta(\cdot |s_t)] \end{bmatrix} $$
- 위 수식과 같이 constraint가 아니라 objective에 penalty를 주는 형태이다. 하지만 다양한 문제에서 모두 잘 동작하는 \\(\beta\\)를 찾는 것이 어렵기 때문에 TRPO에서는 penalty대신에 constraint를 취하는 방식을 선택했다.

# Fisher information matrix

# Conjugate gradient

## Surrogate objective function

- 실제 objective function을 최대화 하기 어렵기 때문에 surrogate objective function을 정의했고, 따라서 surrogate objective function을 update하면서 실제 update하고 싶은 objective function의 update를 기대한다.

- total accumulated reward의 최대화문제를 policy gradient theorem에 의해 expectation으로 변경한 것을 다시 Expectation을 적용한 것이다.<br><br>
$$ L^{CPI}(\theta) = \mathbb{E}_t \left[ \frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}A_t \right] = \mathbb{E}_t[r_t(\theta) A_t] $$

### Clipped surrogate objective

- TRPO의 surrogate function을 강제적으로 clipping하는 방법이다.
- surrogate function
  - \\(r_t(\theta) = \frac{\pi_\theta(a_t\;|\;s_t)}{\pi_{\theta_{old}}(a_t\;|\;s_t)}, \;\; r_t(\theta_{old}) = 1\\)
  - 위 수식을 이용해서 surrogate function을 최대화한다.
  - \\(L^{CPI}(\theta) = \hat{\mathbb{E}}_t \begin{bmatrix} \frac{\pi_\theta(a_t\;|\;s_t)}{\pi_{\theta_{old}}(a_t\;|\;s_t)} \hat{A}_t \end{bmatrix} = \hat{\mathbb{E}}_t \begin{bmatrix} r_t(\theta) \hat{A}_t \end{bmatrix} \\)

- 하지만, 위 수식을 그대로 사용하면 급격한 policy update문제에 봉착하게 되므로 penalty를 이용해서 필요 이상의 policy update를 방지한다.
- TRPO에서는 KL-divergence를 사용하지만, PPO에서는 clipping 방법을 사용한다.<br><br>
$$ L^{CLIP}(\theta) = \hat{\mathbb{E}}_t[ min(r_t(\theta) \hat{A}_t, clip(r_t(\theta), 1 - \epsilon, 1 + \epsilon) \hat{A}_t)] $$<br>
  - 추가적으로, \\(\epsilon\\)은 hyperparameter이고, 0.2일 때 가장 성능이 좋았으며, Atari game에서는 0.1 * \\(\alpha\\)를 사용한다. 여기서 \\(\alpha\\)는 학습률 1에서 시작해서 학습이 진행됨에 따라 0으로 감소한다.
  - Clipped와 unclipped objective중 min값을 선택함으로써 \\(L^{CLIP}(\theta)\\)는 unclipped objective에 대한 lower bound가 된다.

![default](https://user-images.githubusercontent.com/22078438/49698818-5cb56900-fc0c-11e8-8213-68ac1ff6e844.PNG)

- 위 그림은 Advantage function \\(\hat{A}_t\\)의 부호에 따라 clip에 대해서 설명한다.
  - Advantage function이 \\(\hat{A}_t\\)가 양수일 때,
    - Advantage가 현재보다 높다라는 뜻이고 파라미터를 +의 방향으로 update해야 한다. 즉, 어떤 상태 s에서 행동 a가 평균보다 좋다는 의미이다. 따라서 이것을 취할 확률이 증가하게 되고 \\(r_t(\theta)\\)를 clip하여 \\(\epsilon\\)보다 커지지 않도록 유도하는 것이다.
    - 예외적으로, TRPO에서 사용했던 constraint가 아니기 때문에 \\(\pi_\theta(a_t|s_t)\\)의 증가량이 \\(\epsilon\\)보다 커질수도 있다. 하지만 증가량이 더 커지더라도 objective function update에 효과적이지 않을 수 있기 떄문에 clip으로 \\(\epsilon\\)이하로 유지된다.
    - 다른 예외 상황으로, \\(r_t(\theta)\\)가 objective function의 값을 감소 시키는 방향으로 움직이는 경우라도 \\(1-\epsilon\\)보다 작아질 수 있다. clip의 목적은 최대한 lower bound를 구하는 것이기 때문이다.
  <br><br>
  - Advantage function이 음수 일때,
    - Advantage가 현재보다 좋지 않다라는 뜻이며 그 반대방향으로 update해야 한다. 즉, 어떤 상태 s에서 행동 a가 평균보다 좋지 않다는 의미이다. 따라서 이것을 취할 확률이 감소하게 되고, \\(r_t(\theta)\\)를 clip하여 \\(\epsilon\\)보다 작아지지 않도록 유도한다.

# Adaptive KL penalty coefficient

# Surrogate objective 비교

- No clipping or penalty, \\(L_t(\theta) = r_t(\theta) \hat{A}_t\\)
- Clipping, \\(L_t(\theta) = min(r_t(\theta) \hat{A}_t, clip(r_t(\theta), 1 - \epsilon, 1 + \epsilon)\hat{A}_t) \\)
- KL penalty(fixed or adaptive), \\(L_t(\theta) = r_t(\theta) \hat{A}_t - \beta KL[\pi_{\theta_{old}}, \pi_\theta] \\)

# 참조

[PPO 여행](https://reinforcement-learning-kr.github.io/2018/06/22/7_ppo/)

- 위 블로그 참조하였습니다. 감사합니다.